In [1]:
from osgeo import gdal
import wradlib as wrl

file_path = "C:\\Users\\manan\\OneDrive\\Desktop\\data\\BHP190610110229.RAWSU69"
fcontent = wrl.io.read_iris(file_path)
fcontent = fcontent['data']

In [5]:
import math
%matplotlib widget
import matplotlib.pyplot as plt

def convert_radian(angle):
    angle*= math.pi/180
    return angle
def plot(df):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.scatter3D(cart['X'], cart['Y'], cart['Z'], marker = 1)
    fig.show()
    input('hi')

In [6]:
print(type(fcontent))

<class 'collections.OrderedDict'>


In [7]:
import pandas as pd
import numpy as np

for key1 in fcontent:#10 readings at different elevations
    content = fcontent[key1]
    for key2 in content:#6 readings -> metadata,values for dbz,vel,width
        if key2=='sweep_data':#3 reading taken -> values are stored with key 'sweepdata'
            reading = content[key2]
            db_dbz = reading['DB_DBZ']#extract only dbz data from reading
            #db_dbz['azi_start'][0]-=360
            dbz = pd.DataFrame.from_dict({'azi_start':db_dbz['azi_start'],'azi_stop':db_dbz['azi_stop'],'ele_start':db_dbz['ele_start'],'ele_stop':db_dbz['ele_stop']})#convert dbz data to dataframe
            dbz['azi_mean'] = dbz[['azi_start','azi_stop']].mean(axis=1)#compute mean of azi
            dbz['ele_mean'] = dbz[['ele_start','ele_stop']].mean(axis=1)#compute mean of ele
            dbz['azi_mean'] = dbz['azi_mean'].apply(convert_radian)#convert mean from degree to radian
            dbz['ele_mean'] = dbz['ele_mean'].apply(convert_radian)#convert mean from degree to radian
            data = db_dbz['data']#extract data list (2-d) (360x500)
            #print(data)
            #print(data.shape)
            #print(dbz)
            #print(len(data))
            x,y,z,v = [],[],[],[]#declare lists for coordinates and values
            for j in range(len(data)):
                azi = dbz['azi_mean'][j]#read azi for particular ray
                ele = dbz['ele_mean'][j]#read ele for particular ray
                #print('{}    {}'.format(azi,ele))
                dat = data[j]
                for k,d in enumerate(dat):#process all (500) reading of particular ray
                    i = k*0.5
                    Z = i*math.sin(ele)
                    Y = i*math.cos(ele)*math.cos(azi)
                    X = i*math.cos(ele)*math.sin(azi)
                    x.append(X)
                    y.append(Y)
                    z.append(Z)
                    v.append(d)
            cart = pd.DataFrame({'X':x,'Y':y,'Z':z,'V':v})
            print(cart)
            plot(cart)
    break

               X           Y         Z     V
0       0.000000   -0.000000  0.000000 -32.0
1       0.000024   -0.499999  0.001150 -32.0
2       0.000048   -0.999997  0.002301 -32.0
3       0.000072   -1.499996  0.003451 -32.0
4       0.000096   -1.999995  0.004602 -32.0
...          ...         ...       ...   ...
179995 -4.318405  247.461668  0.569490 -32.0
179996 -4.327129  247.961590  0.570640 -32.0
179997 -4.335853  248.461513  0.571791 -32.0
179998 -4.344577  248.961436  0.572941 -32.0
179999 -4.353301  249.461358  0.574092 -32.0

[180000 rows x 4 columns]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hi 
